# Overview

In this notebook, I used Pytorch Lightning to solve it as a multi-label problem.
I used the following [notebook](https://www.kaggle.com/demetrypascal/better-train-csv-format-keras-starter) as a reference.

The accuracy of the multi-label solution is about the same as that of the simple solution, and I think the accuracy can be improved by post-processing.

[Inference Notebook](https://www.kaggle.com/pegasos/plant2021-multi-label-model-inference)

## Version Notes

- V4  Model: Resnet50,           IMAGE_SIZE: 512, BS: 32, LB: 0.616
- V6  Model: SE-ResNeXt50_32x4d, IMAGE_SIZE: 512, BS: 16, LB: 0.555
- V8  Model: Resnet50, IMAGE_SIZE: 512, BS: 32, LB: 0.584
  - Add processing to remove duplicates [Reference Discussion](https://www.kaggle.com/c/plant-pathology-2021-fgvc8/discussion/227829)
- V11 Model: Resnet50, IMAGE_SIZE: 512, BS: 32, LB: 0.585
  - More epoch, change lr_scheduler
- V14 Model: Resnet50, IMAGE_SIZE: 512, BS: 32, LB: 0.572
  - used torchmetrics(F1, weighted)
- V15 Model: Resnet50, IMAGE_SIZE: 512, BS: 32, LB: 0.560
  - Focal Loss(alpha=1, gamma=2)
- V16 Model: Resnet50, IMAGE_SIZE: 512, BS: 32, LB: 0.580
  - iterative-stratification(cross validators with stratification for multilabel data)
- V17 Model: Resnet50, IMAGE_SIZE: 512, BS: 32, LB: 0.758
  - epoch 60
- V18 Model: EfficientNetB5 NS, IMAGE_SIZE: 512, BS: 32, LB: ???
  - change model

In [ ]:
!pip install -q torchmetrics
!pip install -q iterative-stratification
!pip install -q pytorch-lightning==1.2.8

In [ ]:
package_paths = [
    '../input/pytorch-image-library/pytorch-image-models-master/pytorch-image-models-master',
]
import sys;

for pth in package_paths:
    sys.path.append(pth)

import timm

# Import

In [ ]:
import pandas as pd
import numpy as np
import cv2
import timm
import torch
import torch.nn as nn
import albumentations as A
import pytorch_lightning as pl
import matplotlib.pyplot as plt
import torchmetrics

from torch.utils.data import Dataset, DataLoader
from albumentations.core.composition import Compose, OneOf
from albumentations.pytorch import ToTensorV2

from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning import Callback
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import StratifiedKFold

In [ ]:
print(f"PyTorch Lightning version: {pl.__version__}")

# Config

In [ ]:
DEBUG = False

class CFG:
    seed = 42
    model_name = 'tf_efficientnet_b5_ns'
    pretrained = True
    img_size = 512
    num_classes = 6
    lr = 1e-4
    max_lr = 1e-3
    pct_start = 0.3
    div_factor = 1.0e+3
    final_div_factor = 1.0e+3
    num_epochs = 20
    batch_size = 16
    accum = 1
    precision = 16
    n_fold = 5
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
PATH = "../input/plant-pathology-2021-fgvc8/"

# TRAIN_DIR = PATH + 'train_images/'
TRAIN_DIR = "../input/resized-plant2021/img_sz_640/"
TEST_DIR = PATH + 'test_images/'

In [ ]:
seed_everything(CFG.seed)

In [ ]:
df_all = pd.read_csv(PATH + "train.csv")
if DEBUG == True:
    df_all = df_all[:200]
    CFG.num_epochs = 30

df_all.shape

In [ ]:
from collections import defaultdict


dct = defaultdict(list)

for i, label in enumerate(df_all.labels):
    for category in label.split():
        dct[category].append(i)
 
dct = {key: np.array(val) for key, val in dct.items()}
dct

In [ ]:
new_df = pd.DataFrame(np.zeros((df_all.shape[0], len(dct.keys())), dtype=np.int8), columns=dct.keys())

for key, val in dct.items():
    new_df.loc[val, key] = 1

new_df.head()

In [ ]:
df_all = pd.concat([df_all, new_df], axis=1)
df_all.to_csv('better_train.csv', index = False)
df_all.head()

# Remove duplicates

The output of the following [notebook](https://www.kaggle.com/nickuzmenkov/pp2021-duplicates-revealing/output) was used as a reference.
If you find it useful, please vote not only for this notebook, but also for the notebook it refers to!

In [ ]:
duplicates = pd.read_csv("../input/pp2021-duplicates-revealing/duplicates.csv",  names=('image1', 'image2'))
sorted_index = duplicates['image1'].sort_values().index
duplicates = duplicates.iloc[sorted_index].reset_index(drop=True)
duplicates.head()

Duplicate images removed and the label should be the sum of the two.

In [ ]:
if DEBUG != True:
    for idx, images in duplicates.iterrows():
    #     print(images['image1'])
        mask1 = df_all['image'] == images['image1']
        mask2 = df_all['image'] == images['image2']
        tmp = df_all[mask1].iloc[0, 2:].values | df_all[mask2].iloc[0, 2:].values
        df_all.loc[mask1, df_all.columns[2:]] = tmp
        df_all = df_all.drop(df_all[mask2].index)
    assert (len(new_df) - len(duplicates)) == len(df_all)

## Split Train Data

In [ ]:
# sfk = StratifiedKFold(CFG.n_fold)
# for train_idx, valid_idx in sfk.split(df_all['image'], df_all['labels']):
#     df_train = df_all.iloc[train_idx]
#     df_valid = df_all.iloc[valid_idx]
#     break
    
# print(f"train size: {len(df_train)}")
# print(f"valid size: {len(df_valid)}")

# MultilabelStratifiedKFold

In [ ]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold


msss = MultilabelStratifiedKFold(n_splits=CFG.n_fold, shuffle=True, random_state=CFG.seed)

for train_idx, valid_idx in msss.split(df_all['image'], df_all.loc[:, list(df_all.columns[2:].values)]):
    df_train = df_all.iloc[train_idx]
    df_valid = df_all.iloc[valid_idx]

print(f"train size: {len(df_train)}")
print(f"valid size: {len(df_valid)}")

# Define Dataset

In [ ]:
class PlantDataset(Dataset):
    def __init__(self, df, transform=None):
        self.image_id = df['image'].values
        self.labels = df.iloc[:, 2:].values
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image_id = self.image_id[idx]
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        
        image_path = TRAIN_DIR + image_id
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        augmented = self.transform(image=image)
        image = augmented['image']
        return {'image':image, 'target': label}

In [ ]:
def get_transform(phase: str):
    if phase == 'train':
        return Compose([
            A.RandomResizedCrop(height=CFG.img_size, width=CFG.img_size),
            A.Flip(p=0.5),
            A.RandomRotate90(p=0.5),
            A.ShiftScaleRotate(p=0.5),
            A.HueSaturationValue(p=0.5),
            A.OneOf([
                A.RandomBrightnessContrast(p=0.5),
                A.RandomGamma(p=0.5),
            ], p=0.5),
            A.OneOf([
                A.Blur(p=0.1),
                A.GaussianBlur(p=0.1),
                A.MotionBlur(p=0.1),
            ], p=0.1),
            A.OneOf([
                A.GaussNoise(p=0.1),
                A.ISONoise(p=0.1),
                A.GridDropout(ratio=0.5, p=0.2),
                A.CoarseDropout(max_holes=16, min_holes=8, max_height=16, max_width=16, min_height=8, min_width=8, p=0.2)
            ], p=0.2),
            A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])
    else:
        return Compose([
            A.Resize(height=CFG.img_size, width=CFG.img_size),
            A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])

In [ ]:
train_dataset = PlantDataset(df_train, get_transform('train'))
valid_dataset = PlantDataset(df_valid, get_transform('valid'))

train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=True, pin_memory=True, drop_last=True, num_workers=2)
valid_loader = DataLoader(valid_dataset, batch_size=CFG.batch_size, shuffle=False, pin_memory=True, num_workers=2)

In [ ]:
CFG.steps_per_epoch = len(train_loader)
CFG.steps_per_epoch

# Focal Loss

In [ ]:
import torch.nn.functional as F


class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, logits=False, reduce=True):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.logits = logits
        self.reduce = reduce

    def forward(self, inputs, targets):
        if self.logits:
            BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduce=False)
        else:
            BCE_loss = F.binary_cross_entropy(inputs, targets, reduce=False)
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduce:
            return torch.mean(F_loss)
        else:
            return F_loss

# Define Model

In [ ]:
class CustomResNet(nn.Module):
    def __init__(self, model_name='resnet18', pretrained=True):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        in_features = self.model.get_classifier().in_features
#         self.model.fc = nn.Linear(in_features, CFG.num_classes)
        self.model.fc = nn.Sequential(
            nn.Linear(in_features, in_features),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(in_features, CFG.num_classes)
        )

    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
class CustomEffNet(nn.Module):
    def __init__(self, model_name='tf_efficientnet_b0_ns', pretrained=True):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        in_features = self.model.get_classifier().in_features
#         self.model.fc = nn.Linear(in_features, CFG.num_classes)
        self.model.classifier = nn.Sequential(
            nn.Linear(in_features, in_features),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(in_features, CFG.num_classes)
        )

    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
class LitCassava(pl.LightningModule):
    def __init__(self, model):
        super(LitCassava, self).__init__()
        self.model = model
#         self.metric = pl.metrics.F1(num_classes=CFG.num_classes)
        self.metric = torchmetrics.F1(CFG.num_classes, average='weighted')
#         self.criterion = nn.BCELoss()
        self.criterion = nn.BCEWithLogitsLoss()
#         self.criterion = FocalLoss()
        self.sigmoid = nn.Sigmoid()
        self.lr = CFG.lr

    def forward(self, x, *args, **kwargs):
        return self.model(x)

    def configure_optimizers(self):
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr)
#         self.scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(self.optimizer, T_max=CFG.t_max, eta_min=CFG.min_lr)
        self.scheduler = torch.optim.lr_scheduler.OneCycleLR(self.optimizer, 
                                                             epochs=CFG.num_epochs, steps_per_epoch=CFG.steps_per_epoch,
                                                             max_lr=CFG.max_lr, pct_start=CFG.pct_start, 
                                                             div_factor=CFG.div_factor, final_div_factor=CFG.final_div_factor)
        scheduler = {'scheduler': self.scheduler, 'interval': 'step',}

        return [self.optimizer], [scheduler]

    def training_step(self, batch, batch_idx):
        image = batch['image']
        target = batch['target']
        output = self.model(image)
#         output = self.sigmoid(output)
        loss = self.criterion(output, target)
        score = self.metric(self.sigmoid(output), target.clone().detach().to(torch.int32))
        logs = {'train_loss': loss, 'train_f1': score, 'lr': self.optimizer.param_groups[0]['lr']}
        self.log_dict(
            logs,
            on_step=False, on_epoch=True, prog_bar=True, logger=True
        )
        return loss
    
    def validation_step(self, batch, batch_idx):
        image = batch['image']
        target = batch['target']
        output = self.model(image)
#         output = self.sigmoid(output)
        loss = self.criterion(output, target)
        score = self.metric(self.sigmoid(output), target.clone().detach().to(torch.int32))
        logs = {'valid_loss': loss, 'valid_f1': score}
        self.log_dict(
            logs,
            on_step=False, on_epoch=True, prog_bar=True, logger=True
        )
        return loss

In [ ]:
# model = CustomResNet(model_name=CFG.model_name, pretrained=CFG.pretrained)
# lit_model = LitCassava(model.model)

In [ ]:
model = CustomEffNet(model_name=CFG.model_name, pretrained=CFG.pretrained)
lit_model = LitCassava(model.model)

In [ ]:
logger = CSVLogger(save_dir='logs/', name=CFG.model_name)
logger.log_hyperparams(CFG.__dict__)
checkpoint_callback = ModelCheckpoint(monitor='valid_f1',
                                      save_top_k=1,
                                      save_last=True,
                                      save_weights_only=True,
                                      filename='{epoch:02d}-{valid_loss:.4f}-{valid_f1:.4f}',
                                      verbose=False,
                                      mode='max')

trainer = Trainer(
    max_epochs=CFG.num_epochs,
    gpus=[0],
    accumulate_grad_batches=CFG.accum,
    precision=CFG.precision,
#     callbacks=[EarlyStopping(monitor='valid_loss', patience=3, mode='min')],
    checkpoint_callback=checkpoint_callback,
    logger=logger,
    weights_summary='top',
    amp_backend='native',
)

# Training

In [ ]:
trainer.fit(lit_model, train_dataloader=train_loader, val_dataloaders=valid_loader)

# Result

In [ ]:
metrics = pd.read_csv(f'{trainer.logger.log_dir}/metrics.csv')

train_acc = metrics['train_f1'].dropna().reset_index(drop=True)
valid_acc = metrics['valid_f1'].dropna().reset_index(drop=True)
    
fig = plt.figure(figsize=(7, 6))
plt.grid(True)
plt.plot(train_acc, color="r", marker="o", label='train/f1')
plt.plot(valid_acc, color="b", marker="x", label='valid/f1')
plt.ylabel('F1', fontsize=24)
plt.xlabel('Epoch', fontsize=24)
plt.legend(loc='lower right', fontsize=18)
plt.savefig(f'{trainer.logger.log_dir}/f1.png')

train_loss = metrics['train_loss'].dropna().reset_index(drop=True)
valid_loss = metrics['valid_loss'].dropna().reset_index(drop=True)

fig = plt.figure(figsize=(7, 6))
plt.grid(True)
plt.plot(train_loss, color="r", marker="o", label='train/loss')
plt.plot(valid_loss, color="b", marker="x", label='valid/loss')
plt.ylabel('Loss', fontsize=24)
plt.xlabel('Epoch', fontsize=24)
plt.legend(loc='upper right', fontsize=18)
plt.savefig(f'{trainer.logger.log_dir}/loss.png')\

lr = metrics['lr'].dropna().reset_index(drop=True)

fig = plt.figure(figsize=(7, 6))
plt.grid(True)
plt.plot(lr, color="g", marker="o", label='learning rate')
plt.ylabel('LR', fontsize=24)
plt.xlabel('Epoch', fontsize=24)
plt.legend(loc='upper right', fontsize=18)
plt.savefig(f'{trainer.logger.log_dir}/lr.png')